In [9]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from data_process import tokenizers, utils
from nanoGPT.model import GPT, GPTConfig
from training.lightning_training import LightningGPT, GamesDataModule, GamesDataset, test_dataloader
from datasets import load_dataset
import pytorch_lightning as pl
import torch
import os
import numpy as np
import seaborn as sns
from data_process.utils import (
    join_material_tokens,
    remove_material_tokens,
    remove_last_player_material_token,
    row_for_base_training,
)

import training.load_models as load_models
import json

In [41]:
from data_process.utils import add_elo_and_piece_count_to_dataset, add_elo_to_dataset


columns_to_load = [
    #"index",
    #"id",
    #"date",
    "white_elo",
    "black_elo",
    #"result",
    "ply",
    "ply_30s",
    "piece_uci"
]

piece_count_data_files = {"test": "./data/test_piece_count.csv"}
material_data_files = {"test": "./data/test.csv"}

# Load both splits
dataset = load_dataset(
    "csv",
    data_files=piece_count_data_files,
    delimiter=";",
    usecols=columns_to_load,
    num_proc=6,
    # split="test",
)

material_dataset = load_dataset(
    "csv",
    data_files=material_data_files,
    delimiter=";",
    usecols=columns_to_load,
    num_proc=6,
    # split="test",
)

columns_to_remove = [
    #"index",
    #"id",
    #"date",
    "white_elo",
    "black_elo",
    #"result",
    # "ply",
    "ply_30s",
    "piece_uci"
]

no_elo_dataset = dataset.map(row_for_base_training, num_proc=6, remove_columns=["ply_30s", "piece_uci"])
elo_dataset = dataset.map(add_elo_to_dataset, num_proc=6, remove_columns=columns_to_remove)
piece_count_dataset = dataset.map(add_elo_and_piece_count_to_dataset, num_proc=6, remove_columns=columns_to_remove)
material_dataset = material_dataset.map(add_elo_and_piece_count_to_dataset, num_proc=6, remove_columns=columns_to_remove)

no_elo_dataset = no_elo_dataset["test"]#.shuffle(seed=42).select(range(200))
# dataset["test"] = dataset["test"].shuffle(seed=42).select(range(20000))
# material_dataset["test"] = material_dataset["test"].shuffle(seed=42).select(range(20000))
# no_elo_dataset["test"] = no_elo_dataset["test"].shuffle(seed=42).select(range(20000))

In [42]:
from training.model_configs import *

test_start_token = 1
test_token_step = 2

test_start_token_base = test_start_token // 2 + 1
test_token_step_base = 1

material_checkpoint = "./models/full_training/elo_material_ignore_material_prediction/epoch=9-step=1250000.ckpt"
material_model = load_models.material_model(material_checkpoint)

material_pair_checkpoint = "./models/full_training/elo_material_pair_ignore_material_prediction/epoch=9-step=1250000.ckpt"
material_pair_model = load_models.material_pair_model(material_pair_checkpoint)

piece_count_checkpoint = "./models/full_training/elo_piece_count_ignore_material_prediction/epoch=9-step=1250000.ckpt"
piece_count_model = load_models.piece_count_model(piece_count_checkpoint)

base_checkpoint = "./models/full_training/masked_elo/epoch=9-step=1250000.ckpt"
base_model = load_models.base_elo_model(base_checkpoint)

base_no_mask_checkpoint = "./models/full_training/adaptive_elo/epoch=9-step=1250000.ckpt"
base_no_mask_model = load_models.base_elo_no_mask_model(base_no_mask_checkpoint)

no_elo_checkpoint = "./models/full_training/no_elo/epoch=9-step=1250000.ckpt"
no_elo_model = load_models.no_elo_model(no_elo_checkpoint)

number of parameters: 27.34M
number of parameters: 28.15M
number of parameters: 28.17M
number of parameters: 27.29M
number of parameters: 27.29M
number of parameters: 27.27M


In [43]:
test_dl = test_dataloader(no_elo_dataset, tokenizer=no_elo_model.tokenizer, batch_size=16)

In [45]:
trainer = pl.Trainer(
    logger=False,
    accelerator="gpu",
    max_epochs=10,
    # callbacks=[pl.callbacks.RichProgressBar()],
    # precision="bf16-mixed",
)
# torch.set_float32_matmul_precision('high')

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [46]:
batch = next(iter(test_dl))
batch

{'input_ids': tensor([[   1, 3689, 3892,  ...,    0,    0,    0],
         [   1, 3689, 3670,  ...,    0,    0,    0],
         [   1, 3689, 3892,  ...,    0,    0,    0],
         ...,
         [   1, 3361, 3892,  ...,    0,    0,    0],
         [   1, 3679, 3896,  ...,    0,    0,    0],
         [   1, 3693, 3890,  ...,    0,    0,    0]]),
 'target_ids': tensor([[3689, 3892, 3343,  ...,    0,    0,    0],
         [3689, 3670, 3685,  ...,    0,    0,    0],
         [3689, 3892,   23,  ...,    0,    0,    0],
         ...,
         [3361, 3892, 3685,  ...,    0,    0,    0],
         [3679, 3896, 2793,  ...,    0,    0,    0],
         [3693, 3890, 3361,  ...,    0,    0,    0]]),
 'white_elo': [2500,
  2600,
  2700,
  2600,
  2600,
  2700,
  2500,
  2700,
  2700,
  2700,
  2700,
  2600,
  2600,
  2600,
  2600,
  2700],
 'black_elo': [2600,
  2500,
  2600,
  2600,
  2500,
  2600,
  2600,
  2600,
  2500,
  2600,
  2500,
  2600,
  2500,
  2700,
  2500,
  2600]}

In [47]:
no_elo_model_test_result = trainer.test(no_elo_model, dataloaders=test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5100522637367249     │
│     test_acc_elo_1000     │    0.4861624836921692     │
│  test_acc_elo_1000_1000   │    0.48291170597076416    │
│  test_acc_elo_1000_1100   │    0.4848529100418091     │
│  test_acc_elo_1000_1200   │     0.488752543926239     │
│  test_acc_elo_1000_1300   │    0.5040000081062317     │
│  test_acc_elo_1000_1400   │     0.380952388048172     │
│  test_acc_elo_1000_1500   │    0.6296296119689941     │
│  test_acc_elo_1000_1600   │    0.5760869383811951     │
│  test_acc_elo_1000_1700   │    0.5357142686843872     │
│  test_acc_elo_1000_1800   │    0.5454545617103577     │
│  test_acc_elo_1000_1900   │    0.5098039507865906     │
│  test_acc_elo_1000_2100   │    0.3333333432674408     │
│   test_acc_elo_1000_400   │    0.4655172526836395     │
│   test_acc_elo_1000_500   │    0.1666666716337204     │
│   test_acc_elo_1000_600   │    0.4333333373069763     │
│   test_acc_elo_1000_700   │    0.4514285624027252     │
│   test_acc_elo_1000_800   │    0.5005662441253662     │
│   test_acc_elo_1000_900   │    0.4954703748226166     │
│     test_acc_elo_1100     │    0.49153169989585876    │
│  test_acc_elo_1100_1000   │    0.49144861102104187    │
│  test_acc_elo_1100_1100   │    0.4963243305683136     │
│  test_acc_elo_1100_1200   │    0.4871700406074524     │
│  test_acc_elo_1100_1300   │    0.48181816935539246    │
│  test_acc_elo_1100_1400   │    0.5731707215309143     │
│  test_acc_elo_1100_1500   │    0.4399999976158142     │
│  test_acc_elo_1100_1600   │    0.42276424169540405    │
│  test_acc_elo_1100_1700   │    0.47058823704719543    │
│  test_acc_elo_1100_1800   │    0.3076923191547394     │
│   test_acc_elo_1100_400   │    0.40740740299224854    │
│   test_acc_elo_1100_500   │    0.6153846383094788     │
│   test_acc_elo_1100_600   │    0.1666666716337204     │
│   test_acc_elo_1100_700   │    0.4404761791229248     │
│   test_acc_elo_1100_800   │    0.4309859275817871     │
│   test_acc_elo_1100_900   │    0.45886075496673584    │
│     test_acc_elo_1200     │    0.49668949842453003    │
│  test_acc_elo_1200_1000   │    0.49087220430374146    │
│  test_acc_elo_1200_1100   │    0.49564042687416077    │
│  test_acc_elo_1200_1200   │    0.5007259249687195     │
│  test_acc_elo_1200_1300   │    0.4910222887992859     │
│  test_acc_elo_1200_1400   │    0.5322997570037842     │
│  test_acc_elo_1200_1500   │    0.44011977314949036    │
│  test_acc_elo_1200_1600   │    0.36000001430511475    │
│  test_acc_elo_1200_1700   │    0.4126984179019928     │
│  test_acc_elo_1200_1800   │    0.4098360538482666     │
│  test_acc_elo_1200_2200   │            0.5            │
│   test_acc_elo_1200_400   │    0.3478260934352875     │
│   test_acc_elo_1200_500   │    0.20000000298023224    │
│   test_acc_elo_1200_600   │    0.37931033968925476    │
│   test_acc_elo_1200_700   │     0.692307710647583     │
│   test_acc_elo_1200_800   │    0.47096773982048035    │
│   test_acc_elo_1200_900   │    0.5396825671195984     │
│     test_acc_elo_1300     │    0.4970445930957794     │
│  test_acc_elo_1300_1000   │    0.4939759075641632     │
│  test_acc_elo_1300_1100   │    0.4583333432674408     │
│  test_acc_elo_1300_1200   │    0.49841177463531494    │
│  test_acc_elo_1300_1300   │    0.49966803193092346    │
│  test_acc_elo_1300_1400   │    0.4926547110080719     │
│  test_acc_elo_1300_1500   │    0.44279661774635315    │
│  test_acc_elo_1300_1600   │    0.47311827540397644    │
│  test_acc_elo_1300_1700   │    0.4555555582046509     │
│  test_acc_elo_1300_1800   │    0.4920634925365448     │
│  test_acc_elo_1300_1900   │    0.5662650465965271     │
│  test_acc_elo_1300_2200   │    0.5806451439857483     │
│   test_acc_elo_1300_500   │    0.3333333432674408     │
│   test_acc_elo_1300_600   │    0.47311827540397644    

In [49]:
with open("test_results/accuracy/no_elo_model.json", "w") as f:
    json.dump(no_elo_model_test_result, f)

In [9]:
# data_module = GamesDataModule(datasets=no_elo_dataset, tokenizer=no_elo_tokenizer, num_workers=12, mask_elo_token=False, batch_size=16)

In [10]:
# trainer.test(no_elo_model, datamodule=data_module)

In [13]:
data_module = GamesDataModule(datasets=piece_count_dataset, tokenizer=piece_count_tokenizer, num_workers=12, mask_elo_token=False, batch_size=16)

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [14]:
test_result = trainer.test(piece_count_model, datamodule=data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.521868884563446     │
│     test_acc_elo_1000     │     0.492029070854187     │
│  test_acc_elo_1000_1000   │    0.4917294979095459     │
│  test_acc_elo_1000_1100   │    0.4848529100418091     │
│  test_acc_elo_1000_1200   │    0.5194274187088013     │
│  test_acc_elo_1000_1300   │    0.47200000286102295    │
│  test_acc_elo_1000_1400   │    0.4285714328289032     │
│  test_acc_elo_1000_1500   │    0.6296296119689941     │
│  test_acc_elo_1000_1600   │     0.52173912525177      │
│  test_acc_elo_1000_1700   │    0.4642857015132904     │
│  test_acc_elo_1000_1800   │    0.5454545617103577     │
│  test_acc_elo_1000_1900   │    0.5098039507865906     │
│  test_acc_elo_1000_2100   │    0.3333333432674408     │
│   test_acc_elo_1000_400   │    0.4655172526836395     │
│   test_acc_elo_1000_500   │            0.5            │
│   test_acc_elo_1000_600   │    0.4055555462837219     │
│   test_acc_elo_1000_700   │    0.4571428596973419     │
│   test_acc_elo_1000_800   │     0.524348795413971     │
│   test_acc_elo_1000_900   │    0.4987804889678955     │
│     test_acc_elo_1100     │    0.5021634101867676     │
│  test_acc_elo_1100_1000   │    0.4990498423576355     │
│  test_acc_elo_1100_1100   │    0.5092435479164124     │
│  test_acc_elo_1100_1200   │    0.4968692362308502     │
│  test_acc_elo_1100_1300   │    0.48363634943962097    │
│  test_acc_elo_1100_1400   │    0.5182926654815674     │
│  test_acc_elo_1100_1500   │    0.47999998927116394    │
│  test_acc_elo_1100_1600   │    0.4146341383457184     │
│  test_acc_elo_1100_1700   │    0.4117647111415863     │
│  test_acc_elo_1100_1800   │    0.28205129504203796    │
│   test_acc_elo_1100_400   │    0.48148149251937866    │
│   test_acc_elo_1100_500   │    0.5384615659713745     │
│   test_acc_elo_1100_600   │    0.3333333432674408     │
│   test_acc_elo_1100_700   │    0.3928571343421936     │
│   test_acc_elo_1100_800   │    0.4422535300254822     │
│   test_acc_elo_1100_900   │    0.4651898741722107     │
│     test_acc_elo_1200     │    0.5070490837097168     │
│  test_acc_elo_1200_1000   │    0.5354969501495361     │
│  test_acc_elo_1200_1100   │     0.507675290107727     │
│  test_acc_elo_1200_1200   │    0.5115590691566467     │
│  test_acc_elo_1200_1300   │    0.49666956067085266    │
│  test_acc_elo_1200_1400   │    0.5478036403656006     │
│  test_acc_elo_1200_1500   │    0.4640718698501587     │
│  test_acc_elo_1200_1600   │    0.3400000035762787     │
│  test_acc_elo_1200_1700   │    0.3333333432674408     │
│  test_acc_elo_1200_1800   │    0.4590163826942444     │
│  test_acc_elo_1200_2200   │    0.4583333432674408     │
│   test_acc_elo_1200_400   │    0.3913043439388275     │
│   test_acc_elo_1200_500   │    0.30000001192092896    │
│   test_acc_elo_1200_600   │    0.4137931168079376     │
│   test_acc_elo_1200_700   │    0.6538461446762085     │
│   test_acc_elo_1200_800   │    0.49677419662475586    │
│   test_acc_elo_1200_900   │    0.5264550447463989     │
│     test_acc_elo_1300     │    0.5049799680709839     │
│  test_acc_elo_1300_1000   │    0.5060241222381592     │
│  test_acc_elo_1300_1100   │    0.4800724685192108     │
│  test_acc_elo_1300_1200   │    0.5072191953659058     │
│  test_acc_elo_1300_1300   │    0.5075981020927429     │
│  test_acc_elo_1300_1400   │    0.4966612160205841     │
│  test_acc_elo_1300_1500   │    0.47033897042274475    │
│  test_acc_elo_1300_1600   │    0.47311827540397644    │
│  test_acc_elo_1300_1700   │    0.4611110985279083     │
│  test_acc_elo_1300_1800   │    0.5396825671195984     │
│  test_acc_elo_1300_1900   │    0.5180723071098328     │
│  test_acc_elo_1300_2200   │    0.5806451439857483     │
│   test_acc_elo_1300_500   │    0.3333333432674408     │
│   test_acc_elo_1300_600   │    0.5806451439857483     

In [ ]:
# save test result to file
with open("test_results/piece_count_model.json", "w") as f:
    json.dump(test_result, f)

In [16]:
data_module = GamesDataModule(datasets=material_dataset, tokenizer=material_tokenizer, num_workers=12, mask_elo_token=False, batch_size=16)

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [17]:
test_result = trainer.test(material_model, datamodule=data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5210025310516357     │
│     test_acc_elo_1000     │    0.4957594573497772     │
│  test_acc_elo_1000_1000   │    0.49270251393318176    │
│  test_acc_elo_1000_1100   │    0.4971461892127991     │
│  test_acc_elo_1000_1200   │     0.511247456073761     │
│  test_acc_elo_1000_1300   │    0.4560000002384186     │
│  test_acc_elo_1000_1400   │     0.380952388048172     │
│  test_acc_elo_1000_1500   │    0.5925925970077515     │
│  test_acc_elo_1000_1600   │     0.554347813129425     │
│  test_acc_elo_1000_1700   │    0.4642857015132904     │
│  test_acc_elo_1000_1800   │    0.5454545617103577     │
│  test_acc_elo_1000_1900   │    0.4901960790157318     │
│  test_acc_elo_1000_2100   │    0.2777777910232544     │
│   test_acc_elo_1000_400   │    0.47413793206214905    │
│   test_acc_elo_1000_500   │            0.5            │
│   test_acc_elo_1000_600   │    0.44999998807907104    │
│   test_acc_elo_1000_700   │    0.46857142448425293    │
│   test_acc_elo_1000_800   │    0.5039637684822083     │
│   test_acc_elo_1000_900   │     0.504529595375061     │
│     test_acc_elo_1100     │    0.5036221742630005     │
│  test_acc_elo_1100_1000   │    0.5029966235160828     │
│  test_acc_elo_1100_1100   │    0.5083300471305847     │
│  test_acc_elo_1100_1200   │    0.5005524754524231     │
│  test_acc_elo_1100_1300   │    0.4745454490184784     │
│  test_acc_elo_1100_1400   │    0.5548780560493469     │
│  test_acc_elo_1100_1500   │    0.4320000112056732     │
│  test_acc_elo_1100_1600   │    0.42276424169540405    │
│  test_acc_elo_1100_1700   │     0.529411792755127     │
│  test_acc_elo_1100_1800   │    0.3589743673801422     │
│   test_acc_elo_1100_400   │    0.6296296119689941     │
│   test_acc_elo_1100_500   │    0.5769230723381042     │
│   test_acc_elo_1100_600   │           0.25            │
│   test_acc_elo_1100_700   │    0.4404761791229248     │
│   test_acc_elo_1100_800   │    0.4563380181789398     │
│   test_acc_elo_1100_900   │     0.474683552980423     │
│     test_acc_elo_1200     │    0.5051940679550171     │
│  test_acc_elo_1200_1000   │    0.5212981700897217     │
│  test_acc_elo_1200_1100   │     0.500798225402832     │
│  test_acc_elo_1200_1200   │    0.5108889937400818     │
│  test_acc_elo_1200_1300   │    0.49768316745758057    │
│  test_acc_elo_1200_1400   │     0.552971601486206     │
│  test_acc_elo_1200_1500   │    0.47604790329933167    │
│  test_acc_elo_1200_1600   │    0.38999998569488525    │
│  test_acc_elo_1200_1700   │     0.380952388048172     │
│  test_acc_elo_1200_1800   │    0.4098360538482666     │
│  test_acc_elo_1200_2200   │           0.375           │
│   test_acc_elo_1200_400   │    0.3478260934352875     │
│   test_acc_elo_1200_500   │    0.30000001192092896    │
│   test_acc_elo_1200_600   │    0.37931033968925476    │
│   test_acc_elo_1200_700   │    0.6153846383094788     │
│   test_acc_elo_1200_800   │    0.47096773982048035    │
│   test_acc_elo_1200_900   │    0.5291005373001099     │
│     test_acc_elo_1300     │    0.5036302804946899     │
│  test_acc_elo_1300_1000   │    0.5020080208778381     │
│  test_acc_elo_1300_1100   │     0.47826087474823      │
│  test_acc_elo_1300_1200   │    0.5024545192718506     │
│  test_acc_elo_1300_1300   │    0.5074505805969238     │
│  test_acc_elo_1300_1400   │    0.4939902126789093     │
│  test_acc_elo_1300_1500   │    0.4788135588169098     │
│  test_acc_elo_1300_1600   │    0.46236559748649597    │
│  test_acc_elo_1300_1700   │    0.4333333373069763     │
│  test_acc_elo_1300_1800   │    0.5714285969734192     │
│  test_acc_elo_1300_1900   │    0.5542168617248535     │
│  test_acc_elo_1300_2200   │    0.5483871102333069     │
│   test_acc_elo_1300_500   │    0.3333333432674408     │
│   test_acc_elo_1300_600   │    0.5913978219032288     

In [ ]:
with open("test_results/material_model.json", "w") as f:
    json.dump(test_result, f)

In [23]:
test_result_3 = trainer.test(material_pair_model, datamodule=material_pair_data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5183427333831787     │
└───────────────────────────┴───────────────────────────┘

In [24]:
material_pair_model.masked_elo_test = False
test_result_4 = trainer.test(material_pair_model, datamodule=material_pair_data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5111902952194214     │
│         test_loss         │    1.5219067335128784     │
└───────────────────────────┴───────────────────────────┘

In [20]:
base_data_module = GamesDataModule(datasets=elo_dataset, tokenizer=piece_count_tokenizer, num_workers=12, mask_elo_token=False, batch_size=16)
test_result_5 = trainer.test(base_model, datamodule=base_data_module)[0]

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5132776498794556     │
│     test_acc_elo_1000     │    0.4854244589805603     │
│  test_acc_elo_1000_1000   │    0.4871261417865753     │
│  test_acc_elo_1000_1100   │    0.4981086552143097     │
│  test_acc_elo_1000_1200   │            0.5            │
│  test_acc_elo_1000_1300   │    0.4727272689342499     │
│  test_acc_elo_1000_1400   │    0.6470588445663452     │
│  test_acc_elo_1000_1500   │     0.47826087474823      │
│  test_acc_elo_1000_1600   │    0.5060241222381592     │
│  test_acc_elo_1000_1700   │    0.4583333432674408     │
│  test_acc_elo_1000_1800   │    0.4285714328289032     │
│  test_acc_elo_1000_1900   │    0.44680851697921753    │
│  test_acc_elo_1000_2100   │            0.5            │
│   test_acc_elo_1000_400   │    0.3762376308441162     │
│   test_acc_elo_1000_500   │            0.0            │
│   test_acc_elo_1000_600   │    0.4503311216831207     │
│   test_acc_elo_1000_700   │    0.37956205010414124    │
│   test_acc_elo_1000_800   │    0.4401595890522003     │
│   test_acc_elo_1000_900   │    0.4777243137359619     │
│     test_acc_elo_1100     │    0.49545007944107056    │
│  test_acc_elo_1100_1000   │    0.48535996675491333    │
│  test_acc_elo_1100_1100   │    0.5018617510795593     │
│  test_acc_elo_1100_1200   │    0.5035115480422974     │
│  test_acc_elo_1100_1300   │    0.4834710657596588     │
│  test_acc_elo_1100_1400   │    0.4693877696990967     │
│  test_acc_elo_1100_1500   │    0.4732142984867096     │
│  test_acc_elo_1100_1600   │    0.5377358198165894     │
│  test_acc_elo_1100_1700   │    0.4615384638309479     │
│  test_acc_elo_1100_1800   │    0.37142857909202576    │
│   test_acc_elo_1100_400   │    0.3636363744735718     │
│   test_acc_elo_1100_500   │    0.23529411852359772    │
│   test_acc_elo_1100_600   │            0.5            │
│   test_acc_elo_1100_700   │    0.3142857253551483     │
│   test_acc_elo_1100_800   │    0.4069400727748871     │
│   test_acc_elo_1100_900   │    0.4155018925666809     │
│     test_acc_elo_1200     │    0.49635010957717896    │
│  test_acc_elo_1200_1000   │    0.49875929951667786    │
│  test_acc_elo_1200_1100   │    0.48864614963531494    │
│  test_acc_elo_1200_1200   │    0.5001314282417297     │
│  test_acc_elo_1200_1300   │    0.49865591526031494    │
│  test_acc_elo_1200_1400   │     0.540123462677002     │
│  test_acc_elo_1200_1500   │    0.4639175236225128     │
│  test_acc_elo_1200_1600   │    0.5662650465965271     │
│  test_acc_elo_1200_1700   │    0.4067796468734741     │
│  test_acc_elo_1200_1800   │    0.4038461446762085     │
│  test_acc_elo_1200_2200   │    0.3499999940395355     │
│   test_acc_elo_1200_400   │    0.5555555820465088     │
│   test_acc_elo_1200_500   │    0.6000000238418579     │
│   test_acc_elo_1200_600   │    0.3333333432674408     │
│   test_acc_elo_1200_700   │          0.3125           │
│   test_acc_elo_1200_800   │    0.4051724076271057     │
│   test_acc_elo_1200_900   │     0.495356023311615     │
│     test_acc_elo_1300     │    0.5031876564025879     │
│  test_acc_elo_1300_1000   │    0.47685185074806213    │
│  test_acc_elo_1300_1100   │    0.4616977274417877     │
│  test_acc_elo_1300_1200   │    0.49101293087005615    │
│  test_acc_elo_1300_1300   │     0.505845308303833     │
│  test_acc_elo_1300_1400   │    0.5118312835693359     │
│  test_acc_elo_1300_1500   │    0.5111111402511597     │
│  test_acc_elo_1300_1600   │    0.47647058963775635    │
│  test_acc_elo_1300_1700   │    0.4591194987297058     │
│  test_acc_elo_1300_1800   │    0.4909090995788574     │
│  test_acc_elo_1300_1900   │    0.5479452013969421     │
│  test_acc_elo_1300_2200   │    0.5185185074806213     │
│   test_acc_elo_1300_500   │    0.4615384638309479     │
│   test_acc_elo_1300_600   │    0.5421686768531799     

In [22]:
with open("test_results/base_elo_model.json", "w") as f:
    json.dump(test_result_5, f)

In [ ]:
base_no_mask_model.masked_elo_test = False
test_result_7 = trainer.test(base_no_mask_model, datamodule=base_data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5128018856048584     │
└───────────────────────────┴───────────────────────────┘

In [28]:
base_no_mask_model.tokenizer

In [31]:
test_result_base_no_mask = trainer.test(base_no_mask_model, datamodule=base_data_module)[0]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5130200386047363     │
│     test_acc_elo_1000     │    0.4830108880996704     │
│  test_acc_elo_1000_1000   │    0.48927778005599976    │
│  test_acc_elo_1000_1100   │    0.4917469024658203     │
│  test_acc_elo_1000_1200   │    0.48500001430511475    │
│  test_acc_elo_1000_1300   │    0.48181816935539246    │
│  test_acc_elo_1000_1400   │    0.6470588445663452     │
│  test_acc_elo_1000_1500   │    0.5652173757553101     │
│  test_acc_elo_1000_1600   │    0.5301204919815063     │
│  test_acc_elo_1000_1700   │    0.2916666567325592     │
│  test_acc_elo_1000_1800   │    0.5714285969734192     │
│  test_acc_elo_1000_1900   │    0.5319148898124695     │
│  test_acc_elo_1000_2100   │    0.4285714328289032     │
│   test_acc_elo_1000_400   │    0.2772277295589447     │
│   test_acc_elo_1000_500   │            0.0            │
│   test_acc_elo_1000_600   │    0.41721853613853455    │
│   test_acc_elo_1000_700   │    0.37956205010414124    │
│   test_acc_elo_1000_800   │    0.4202127754688263     │
│   test_acc_elo_1000_900   │    0.47207698225975037    │
│     test_acc_elo_1100     │    0.4942328929901123     │
│  test_acc_elo_1100_1000   │    0.49087151885032654    │
│  test_acc_elo_1100_1100   │    0.5015557408332825     │
│  test_acc_elo_1100_1200   │    0.49534183740615845    │
│  test_acc_elo_1100_1300   │    0.44628098607063293    │
│  test_acc_elo_1100_1400   │     0.523809552192688     │
│  test_acc_elo_1100_1500   │    0.5267857313156128     │
│  test_acc_elo_1100_1600   │    0.5377358198165894     │
│  test_acc_elo_1100_1700   │    0.6153846383094788     │
│  test_acc_elo_1100_1800   │    0.37142857909202576    │
│   test_acc_elo_1100_400   │    0.3181818127632141     │
│   test_acc_elo_1100_500   │    0.23529411852359772    │
│   test_acc_elo_1100_600   │           0.375           │
│   test_acc_elo_1100_700   │    0.3285714387893677     │
│   test_acc_elo_1100_800   │    0.38485804200172424    │
│   test_acc_elo_1100_900   │    0.4142312705516815     │
│     test_acc_elo_1200     │    0.49996650218963623    │
│  test_acc_elo_1200_1000   │    0.5310173630714417     │
│  test_acc_elo_1200_1100   │    0.49195170402526855    │
│  test_acc_elo_1200_1200   │    0.5017739534378052     │
│  test_acc_elo_1200_1300   │    0.5048723220825195     │
│  test_acc_elo_1200_1400   │    0.5462962985038757     │
│  test_acc_elo_1200_1500   │    0.46048110723495483    │
│  test_acc_elo_1200_1600   │    0.5783132314682007     │
│  test_acc_elo_1200_1700   │    0.35593220591545105    │
│  test_acc_elo_1200_1800   │    0.5192307829856873     │
│  test_acc_elo_1200_2200   │    0.4000000059604645     │
│   test_acc_elo_1200_400   │    0.5555555820465088     │
│   test_acc_elo_1200_500   │    0.6000000238418579     │
│   test_acc_elo_1200_600   │           0.375           │
│   test_acc_elo_1200_700   │           0.25            │
│   test_acc_elo_1200_800   │    0.37068966031074524    │
│   test_acc_elo_1200_900   │    0.5201238393783569     │
│     test_acc_elo_1300     │    0.5028620958328247     │
│  test_acc_elo_1300_1000   │    0.44907405972480774    │
│  test_acc_elo_1300_1100   │    0.48033127188682556    │
│  test_acc_elo_1300_1200   │    0.4920208156108856     │
│  test_acc_elo_1300_1300   │    0.5055433511734009     │
│  test_acc_elo_1300_1400   │    0.5089163184165955     │
│  test_acc_elo_1300_1500   │    0.48641976714134216    │
│  test_acc_elo_1300_1600   │    0.43529412150382996    │
│  test_acc_elo_1300_1700   │    0.4779874086380005     │
│  test_acc_elo_1300_1800   │    0.4363636374473572     │
│  test_acc_elo_1300_1900   │    0.6164383292198181     │
│  test_acc_elo_1300_2200   │    0.7037037014961243     │
│   test_acc_elo_1300_500   │    0.6153846383094788     │
│   test_acc_elo_1300_600   │    0.6144578456878662     

In [32]:
with open("test_results/base_elo_model_no_mask.json", "w") as f:
    json.dump(test_result_base_no_mask, f)